In [40]:
from __future__ import annotations

import numpy as np
import pandas as pd
import requests

import matplotlib.pyplot as plt
import seaborn as sns

import smtplib

## Testing Alpha Vantage API

In [2]:
from alpha_vantage.timeseries import TimeSeries

ts = TimeSeries(key='UX2OT39HH6HK14LB')
data, meta_data = ts.get_intraday('AAPL')

In [38]:
import numpy as np
import pandas as pd
import requests
import time

class AlphaVantageDataLoader():
    def __init__(
                 self, 
                 key: str):
        self.key=key

    
    def load_stock_data(self,
        tickers: list = [],
        frequency: str = 'daily', 
        output_size: str = 'full'
        ) -> dict: 

        freq_dict = {'intraday': 'TIME_SERIES_INTRADAY',
                     'daily': 'TIME_SERIES_DAILY',
                     'weekly': 'TIME_SERIES_WEEKLY',
                     'monthly': 'TIME_SERIES_MONTHLY'}
        
        if type(tickers) == str:
            tickers = [tickers]

        start = time.time()
        meta_dict = {} 
        df_dict = {}
        for num, ticker in enumerate(tickers):
            link = f'https://www.alphavantage.co/query?function={freq_dict[frequency]}&symbol={ticker}&outputsize={output_size}&apikey={self.key}'
            stock = requests.get(link)
            keys_list = list(stock.json().keys())
            data, metadata = stock.json()[keys_list[-1]], stock.json()['Meta Data']

            df = pd.DataFrame(data).T
            df.columns = [col.split(' ')[-1] for col in df.columns]
            df = df.apply(pd.to_numeric)
            
            meta_dict[ticker] = meta_data
            df_dict[ticker]= df

            #Alpha Vantage only allows 5 API call per minute
            if (num+1)%5==0: 
                end = time.time()
                wait_time = (start+60) - end
                time.sleep(wait_time)
                start = time.time()
        
        self.meta = meta_dict

        return df_dict


In [39]:
loader = AlphaVantageDataLoader(key='UX2OT39HH6HK14LB')
stocks =  loader.load_stock_data(tickers = ['AAPL', 'CGX.TO'])

In [26]:
import requests
stock = requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=CGX.TO&outputsize=full&apikey=UX2OT39HH6HK14LB')
data, metadata = stock.json()['Time Series (Daily)'], stock.json()['Meta Data']

In [ ]:
sender_email = "tuateststock@outlook.com" 
rec_email = "wongsangaroon@gmail.com"
password = ("ImTestingStocks123") 
message = "MSFT STOCK ALERT!!! The stock is at above price you set " + "%.6f" % last_price  #The message you want to send